In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

/Users/leo/Desktop/programming/bp_classifier/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
parameters = {
    'learning_rate': 0.001,
    'num_epochs': 30,
    'batch_size': 32,
    'image_size': 32,
    'image_channels': 3,
    'output_size': 10,
    'num_layers': 'na',
    'train_val_split': 0.90,
    'device': 'mps',
    'model_name': 'Basic MobileNetV3',
    'criterion': 'CrossEntropyLoss',
    'optimizer': 'Adam',
    'dataset': 'CIFAR10',
    'best_model_path': 'MN3Small.pt',
}

In [2]:
training_data = 'images'

In [3]:
def load_dataset(data_path, batch_size=32, shuffle=True):
    #load dataset
    transformation = transforms.Compose([transforms.RandomHorizontalFlip(0.2),
                                        transforms.RandomVerticalFlip(0.2),
                                        transforms.RandomRotation(10),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    full_dataset = torchvision.datasets.ImageFolder(root=data_path, transform=transformation)
    print('Number of total images: {}'.format(len(full_dataset)))
    #split dataset into training and validation
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size

    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    print('Number of training images: {}'.format(len(train_dataset)))
    print('Number of test images: {}'.format(len(test_dataset)))

    #create dataloaders
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

5212
